In [1]:
import os, random, copy, cv2, timm, torch
import pandas as pd, numpy as np, torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from timm.loss import LabelSmoothingCrossEntropy

In [2]:


# ============================================================
# 0️⃣ Global Config (v14.1)
# ============================================================
BASE_DIR = "/data/ephemeral/home/data"
TRAIN_META = os.path.join(BASE_DIR, "meta_stage0_10_1_train_v10.csv")
TEST_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_1_test_v10")
SAMPLE_SUB = os.path.join(BASE_DIR, "raw", "sample_submission.csv")
OUT_DIR = "./runs_v14_1_b4_b5"
os.makedirs(OUT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SPECIAL_CLASSES = {3, 7, 14}

# --------------------------
# Configs
# --------------------------
CFG_MAIN = dict(
    model_name="tf_efficientnet_b4_ns",
    num_classes=17,
    img_size=380,
    batch_size=32,
    lr=3e-4,
    weight_decay=1e-4,
    epochs=40,
    patience=8,
    train_ratio=0.8,
    num_workers=4,
    tta_times=20,
)

CFG_SUB = dict(
    model_name="efficientnet_b5.sw_in12k_ft_in1k",
    num_classes=3,
    img_size=448,
    batch_size=16,
    lr=3e-4,
    weight_decay=1e-4,
    epochs=30,
    patience=6,
    train_ratio=0.8,
    num_workers=4,
    tta_times=10,
    target_classes=sorted(list(SPECIAL_CLASSES)),
)

# ============================================================
# 1️⃣ Dataset
# ============================================================
class V10ImageDataset(Dataset):
    def __init__(self, csv_path, transform=None, oversample=False,
                 filter_special=False, for_sub=False, special_classes=SPECIAL_CLASSES):
        df = pd.read_csv(csv_path)
        if filter_special:
            df = df[df["target"].isin(special_classes)].reset_index(drop=True)
        if oversample:
            df = self._oversample(df)
        if for_sub:
            mapping = {c: i for i, c in enumerate(sorted(list(special_classes)))}
            df["sub_target"] = df["target"].map(mapping)
            self.use_sub_target = True
        else:
            self.use_sub_target = False
        self.df = df; self.transform = transform

    def _oversample(self, df):
        factors = {3: 2, 7: 2, 14: 3}
        parts = [df]
        for cls, f in factors.items():
            sub = df[df["target"] == cls]
            if len(sub) > 0 and f > 1:
                parts.append(sub.loc[sub.index.repeat(f - 1)])
        out = pd.concat(parts, axis=0).reset_index(drop=True)
        print("✅ Oversample 완료:", {k: int((out["target"] == k).sum()) for k in factors})
        return out

    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.cvtColor(cv2.imread(row["filepath"]), cv2.COLOR_BGR2RGB)
        label = row["sub_target"] if self.use_sub_target else row["target"]
        if self.transform: img = self.transform(image=img)["image"]
        return img, int(label)

class TestImageDataset(Dataset):
    def __init__(self, sample_csv, test_dir, transform):
        self.df = pd.read_csv(sample_csv) if isinstance(sample_csv, str) else sample_csv.copy()
        self.test_dir, self.transform = test_dir, transform
        self.df["ID"] = self.df["ID"].apply(lambda x: x if str(x).endswith(".jpg") else f"{x}.jpg")
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["ID"]
        img = cv2.cvtColor(cv2.imread(os.path.join(self.test_dir, img_id)), cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)["image"]
        return img, img_id

# ============================================================
# 2️⃣ Transform
# ============================================================
def get_transform(img_size, tta=False):
    base = [
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=(255,255,255)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2(),
    ]
    if not tta:
        return A.Compose([
            A.RandomRotate90(p=0.5),
            A.OneOf([A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5)], p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            A.CoarseDropout(max_holes=4, max_height=img_size//12, max_width=img_size//12,
                            min_holes=1, min_height=img_size//24, min_width=img_size//24,
                            fill_value=255, p=0.2),
        ]+base)
    else:
        return A.Compose([
            A.RandomRotate90(p=1.0),
            A.OneOf([A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5)], p=0.5),
        ]+base)

# ============================================================
# 3️⃣ Train / Eval / Infer
# ============================================================
def create_model(model_name, num_classes, drop=0.2):
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=drop)
    return model.to(device)

def train_one_epoch(model, loader, criterion, optimizer, scaler):
    model.train(); total_loss=0; preds=[]; tgts=[]
    for imgs, lbls in tqdm(loader, desc="[Train]", leave=False):
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            out = model(imgs); loss = criterion(out, lbls)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        total_loss += loss.item()
        preds += out.argmax(1).cpu().tolist(); tgts += lbls.cpu().tolist()
    return total_loss/len(loader), accuracy_score(tgts,preds), f1_score(tgts,preds,average='macro')

@torch.no_grad()
def eval_one_epoch(model, loader, criterion):
    model.eval(); total_loss=0; preds=[]; tgts=[]
    for imgs,lbls in tqdm(loader, desc="[Valid]", leave=False):
        imgs,lbls=imgs.to(device),lbls.to(device)
        with torch.cuda.amp.autocast(True):
            out=model(imgs); loss=criterion(out,lbls)
        total_loss+=loss.item(); preds+=out.argmax(1).cpu().tolist(); tgts+=lbls.cpu().tolist()
    return total_loss/len(loader), accuracy_score(tgts,preds), f1_score(tgts,preds,average='macro')

@torch.no_grad()
def infer_probs(model, loader):
    model.eval(); all_probs=[]; all_ids=[]
    for imgs,ids in tqdm(loader, desc="[Infer]", leave=False):
        imgs=imgs.to(device)
        with torch.cuda.amp.autocast(True): out=F.softmax(model(imgs),dim=1)
        all_probs.append(out.cpu().numpy()); all_ids.extend(ids)
    return np.concatenate(all_probs), all_ids

# ============================================================
# 4️⃣ Train Loop
# ============================================================
def train_loop(cfg, train_ds, valid_ds, model_name, num_classes, model_tag):
    train_loader = DataLoader(train_ds,batch_size=cfg["batch_size"],shuffle=True,num_workers=cfg["num_workers"],pin_memory=True)
    valid_loader = DataLoader(valid_ds,batch_size=cfg["batch_size"],shuffle=False,num_workers=cfg["num_workers"],pin_memory=True)
    model=create_model(model_name,num_classes)
    optimizer=torch.optim.AdamW(model.parameters(),lr=cfg["lr"],weight_decay=cfg["weight_decay"])
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=cfg["epochs"],eta_min=1e-6)
    criterion=LabelSmoothingCrossEntropy(smoothing=0.05); scaler=torch.cuda.amp.GradScaler()
    best_f1=-1; counter=0; best_path=os.path.join(OUT_DIR,f"model_{model_tag}_best.pt")

    for e in range(1,cfg["epochs"]+1):
        print(f"\n==== [{model_tag}] Epoch {e}/{cfg['epochs']} ====")
        tr_l,tr_a,tr_f=train_one_epoch(model,train_loader,criterion,optimizer,scaler)
        va_l,va_a,va_f=eval_one_epoch(model,valid_loader,criterion)
        scheduler.step()
        print(f"[Train] loss={tr_l:.4f} acc={tr_a:.4f} f1={tr_f:.4f}")
        print(f"[Valid] loss={va_l:.4f} acc={va_a:.4f} f1={va_f:.4f}")
        if va_f>best_f1:
            best_f1=va_f; counter=0
            torch.save(model.state_dict(),best_path)
            print(f"✅ Best updated: F1={best_f1:.5f}")
        else:
            counter+=1; print(f"⏳ No improve ({counter}/{cfg['patience']})")
            if counter>=cfg["patience"]: print("🛑 Early stop"); break
    return best_path,best_f1

# ============================================================
# 5️⃣ B4 Seed Sampling
# ============================================================
def train_main_b4_seed_sampling(seeds=[42,43,44]):
    best_ckpt,best_f1=None,-1
    for s in seeds:
        print(f"\n🎲 시드 샘플링 진행 중... SEED={s}")
        random.seed(s); np.random.seed(s)
        torch.manual_seed(s); torch.cuda.manual_seed_all(s)

        full=V10ImageDataset(TRAIN_META,get_transform(CFG_MAIN["img_size"]),oversample=True)
        n_train=int(len(full)*CFG_MAIN["train_ratio"]); n_valid=len(full)-n_train
        tr,v=random_split(full,[n_train,n_valid],generator=torch.Generator().manual_seed(s))
        v=copy.deepcopy(v); v.dataset.transform=get_transform(CFG_MAIN["img_size"],tta=False)

        ckpt,f1=train_loop(CFG_MAIN,tr,v,CFG_MAIN["model_name"],CFG_MAIN["num_classes"],f"main_b4_v14_seed{s}")
        if f1>best_f1:
            best_ckpt,best_f1=ckpt,f1
            print(f"🌟 새로운 최고 F1={best_f1:.5f} @ SEED={s}")
    print(f"\n🏆 최종 선택된 시드 모델: {best_ckpt} | F1={best_f1:.5f}")
    return best_ckpt,best_f1

# ============================================================
# 6️⃣ Sub B5
# ============================================================
def train_sub_b5():
    full=V10ImageDataset(TRAIN_META,get_transform(CFG_SUB["img_size"]),filter_special=True,for_sub=True)
    n_train=int(len(full)*CFG_SUB["train_ratio"]); n_valid=len(full)-n_train
    tr,v=random_split(full,[n_train,n_valid],generator=torch.Generator().manual_seed(42))
    v=copy.deepcopy(v); v.dataset.transform=get_transform(CFG_SUB["img_size"],tta=False)
    return train_loop(CFG_SUB,tr,v,CFG_SUB["model_name"],CFG_SUB["num_classes"],"sub_b5_v14")

# ============================================================
# 7️⃣ 2-Stage Inference
# ============================================================
@torch.no_grad()
def two_stage_infer_v14(main_ckpt, sub_ckpt):
    print("\n🔄 [v14.1] 2-Stage Inference 시작")
    main=create_model(CFG_MAIN["model_name"],CFG_MAIN["num_classes"])
    main.load_state_dict(torch.load(main_ckpt)); main.eval()
    sub=create_model(CFG_SUB["model_name"],CFG_SUB["num_classes"])
    sub.load_state_dict(torch.load(sub_ckpt)); sub.eval()

    base_ds=TestImageDataset(SAMPLE_SUB,TEST_DIR,get_transform(CFG_MAIN["img_size"]))
    base_loader=DataLoader(base_ds,batch_size=CFG_MAIN["batch_size"],shuffle=False)
    main_probs,ids=infer_probs(main,base_loader)

    tta_sum=np.zeros_like(main_probs)
    for i in range(CFG_MAIN["tta_times"]):
        print(f"[Main-TTA] round {i+1}/{CFG_MAIN['tta_times']}")
        tta_ds=TestImageDataset(SAMPLE_SUB,TEST_DIR,get_transform(CFG_MAIN["img_size"],tta=True))
        tta_loader=DataLoader(tta_ds,batch_size=CFG_MAIN["batch_size"],shuffle=False)
        tta_sum+=infer_probs(main,tta_loader)[0]
    main_final=(main_probs+tta_sum/CFG_MAIN["tta_times"])/2
    preds=main_final.argmax(1)

    sub_index_map={i:c for i,c in enumerate(CFG_SUB["target_classes"])}
    special_idx=np.where(np.isin(preds,CFG_SUB["target_classes"]))[0]
    if len(special_idx)>0:
        ids_sel=[ids[i] for i in special_idx]
        sub_df=pd.DataFrame({"ID":ids_sel})
        sub_ds=TestImageDataset(sub_df,TEST_DIR,get_transform(CFG_SUB["img_size"]))
        sub_loader=DataLoader(sub_ds,batch_size=CFG_SUB["batch_size"],shuffle=False)
        sub_probs,_=infer_probs(sub,sub_loader)
        tta_sum_sub=np.zeros_like(sub_probs)
        for i in range(CFG_SUB["tta_times"]):
            print(f"[Sub-TTA] round {i+1}/{CFG_SUB['tta_times']}")
            tta_ds_sub=TestImageDataset(sub_df,TEST_DIR,get_transform(CFG_SUB["img_size"],tta=True))
            tta_loader_sub=DataLoader(tta_ds_sub,batch_size=CFG_SUB["batch_size"],shuffle=False)
            tta_sum_sub+=infer_probs(sub,tta_loader_sub)[0]
        sub_final=(sub_probs+tta_sum_sub/CFG_SUB["tta_times"])/2
        sub_preds_idx=sub_final.argmax(1)
        sub_preds=np.vectorize(sub_index_map.get)(sub_preds_idx)
        for i,j in zip(special_idx,sub_preds): preds[i]=j

    out=pd.DataFrame({"ID":ids,"target":preds})
    path=os.path.join(OUT_DIR,"submission_v14_1_b4_b5.csv")
    out.to_csv(path,index=False)
    print(f"✅ 결과 저장: {path}")
    return path

# ============================================================
# 8️⃣ Entrypoint
# ============================================================
def main_v14_1():
    main_ckpt,f1_main=train_main_b4_seed_sampling(seeds=[42,43,44])
    sub_ckpt,f1_sub=train_sub_b5()
    print(f"\n[훈련 완료] MAIN_F1={f1_main:.4f} | SUB_F1={f1_sub:.4f}")
    two_stage_infer_v14(main_ckpt,sub_ckpt)

if __name__=="__main__":
    main_v14_1()



🎲 시드 샘플링 진행 중... SEED=42
✅ Oversample 완료: {3: 12672, 7: 12928, 14: 9792}


/tmp/ipykernel_640907/869351490.py:102: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=(255,255,255)),
/tmp/ipykernel_640907/869351490.py:111: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=4, max_height=img_size//12, max_width=img_size//12,
/tmp/ipykernel_640907/869351490.py:102: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=(255,255,255)),
/tmp/ipykernel_640907/869351490.py:111: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=4, max_height=img_size//12, max_width=img_size//12,
/data/ephemeral/home/py310/lib/python3.10/site-packages/


==== [main_b4_v14_seed42] Epoch 1/40 ====


[Train]:   0%|                             | 0/2996 [00:00<?, ?it/s]/tmp/ipykernel_640907/869351490.py:133: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
[Valid]:   0%|                              | 0/749 [00:00<?, ?it/s]/tmp/ipykernel_640907/869351490.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
                                                                    

[Train] loss=0.5244 acc=0.9311 f1=0.9439
[Valid] loss=0.3562 acc=0.9914 f1=0.9934
✅ Best updated: F1=0.99340

==== [main_b4_v14_seed42] Epoch 2/40 ====


...

[Sub-TTA] round 7/10


[Sub-TTA] round 8/10


[Sub-TTA] round 9/10


[Sub-TTA] round 10/10


✅ 결과 저장: ./runs_v14_1_b4_b5/submission_v14_1_b4_b5.csv
